# Preprocessing 

In [1]:
import numpy as np
import pandas as pd

## Dataset Specific Preprocessing

### Monitor

[Monitor benchmark dataset](https://github.com/crottyan/mgbench) is used for benchmarking mostly database engines MySQL index creation. As a result the dataset is huge (~7GB) exceeding my hardware limits.

In [2]:
monitor = pd.read_csv('../storage/datasets/monitor_bench1.csv')

In [6]:
# Keep only 10% of the rows
monitor = monitor.iloc[:int(monitor.shape[0]*0.1), :]

# Drop the categorical columns
monitor = monitor.drop(['log_time', 'machine_name', 'machine_group'], axis=1)

# Make sure that we keep only numerical (float) types
print(monitor.dtypes)

# Store it without the default pandas index and without the column names
monitor.to_csv('../storage/datasets/monitor_0_1_fraction.csv', index=False, header=False)